In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
@author: pho
"""
%load_ext autoreload
%autoreload 2
import sys
import importlib
from threading import Thread
import time # for time.sleep
from ipygany import PolyMesh, Scene, IsoColor, WarpByScalar
import pyvista as pv
# from pyvista import _vtk
import pyvistaqt as pvqt
import colorcet as cc # Colormaps:
import numpy as np
import h5py
import hdf5storage # conda install hdf5storage
from pathlib import Path
import bqplot.scales
import seaborn as sns
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.colors import Normalize
# import mplcursors
import math # For color map generation
from matplotlib.colors import ListedColormap
from matplotlib.cm import hsv

import ipywidgets as widgets
# from PyQt5 import QtWidgets, uic
from pyvistaqt import QtInteractor, MainWindow
# from pyqt6 import QApplication
from IPython.external.qt_for_kernel import QtGui
from PyQt5.QtWidgets import QApplication

import datetime as dt
import panel as pn # for interactive widgets
pn.extension()

In [2]:
# import PhoPositionalData as pdp
# from PhoPositionalData import load_exported, process_data
from PhoPositionalData.load_exported import *
# from PhoPositionalData.process_data import process_positionalAnalysis_data, gen_2d_histrogram, get_heatmap_color_vectors, process_chunk_equal_poritions_data, extract_spike_timeseries
from PhoPositionalData.process_data import *
from PhoPositionalData.plot_data import *
from PhoPositionalData.plotting.animations import * # make_mp4_from_plotter
from PhoPositionalData.import_data import * # build_spike_positions_list, build_cellID_reverse_lookup_map
from PhoPositionalData.analysis.interactive_placeCell_config import InteractivePlaceCellConfig, VideoOutputModeConfig, PlottingConfig, PlacefieldComputationParameters  # VideoOutputModeConfig, InteractivePlaceCellConfigs
from PhoPositionalData.analysis.interactive_placeCell_config import print_subsession_neuron_differences

In [3]:
# NeuroPy (Diba Lab Python Repo) Loading
try:
    from neuropy import core
    importlib.reload(core)
except ImportError:
    sys.path.append(r'C:\Users\Pho\repos\NeuroPy') # Windows
    # sys.path.append('/home/pho/repo/BapunAnalysis2021/NeuroPy') # Linux
    # sys.path.append(r'/Users/pho/repo/Python Projects/NeuroPy') # MacOS
    print('neuropy module not found, adding directory to sys.path. \n >> Updated sys.path.')
    from neuropy import core
from neuropy.core.dataSession import SessionConfig, DataSessionLoader, DataSession, processDataSession
from neuropy.core.epoch import NamedEpoch

## Data must be pre-processed using the MATLAB script located here: 
# R:\data\KDIBA\gor01\one\IIDataMat_Export_ToPython_2021_11_23.m

# From pre-computed .mat files:
## 07: 
basedir = r'R:\data\KDIBA\gor01\one\2006-6-07_11-26-53'

# ## 08:
# basedir = r'R:\data\KDIBA\gor01\one\2006-6-08_14-26-15'

neuropy module not found, adding directory to sys.path. 
 >> Updated sys.path.


In [4]:
# KDiba Old Format:
sess = DataSessionLoader.kdiba_old_format_session(r'R:\data\KDIBA\gor01\one\2006-6-07_11-26-53')
active_sess_config = sess.config
session_name = sess.name

	 basepath: R:\data\KDIBA\gor01\one\2006-6-07_11-26-53
	 session_name: 2006-6-07_11-26-53
Loading matlab import file: R:\data\KDIBA\gor01\one\2006-6-07_11-26-53\2006-6-07_11-26-53.spikeII.mat...
done.
Loading matlab import file: R:\data\KDIBA\gor01\one\2006-6-07_11-26-53\2006-6-07_11-26-53.epochs_info.mat...
done.
Loading matlab import file: R:\data\KDIBA\gor01\one\2006-6-07_11-26-53\2006-6-07_11-26-53.position_info.mat...
done.
computing linear positions for all active epochs for session...
Saving updated position results to R:\data\KDIBA\gor01\one\2006-6-07_11-26-53\2006-6-07_11-26-53.position.npy...
2006-6-07_11-26-53.position.npy saved
done.

session.position linear positions computed!


In [5]:
sess.epochs.to_dataframe()

,start,stop,label,duration
0,0.000000,1739.153364,maze1,1739.153364
1,1739.153364,1932.420005,maze2,193.266641


In [6]:
## NOTE: The only place that I need to be careful with indexing is with sess.position properties, as these appear to be represented in terms of the number of 60Hz samples instead of in seconds like the Neurons and other classes.
# active_epoch_name = 'maze1'
# active_epoch_name = 'maze2'

# active_epoch = NamedEpoch(
#         name=active_epoch_name,
#         start_end_times=sess.epochs[active_epoch_name]
#     )

active_epoch_name = 'maze'
active_epoch = NamedEpoch(
        name=active_epoch_name,
        start_end_times=[sess.epochs['maze1'][0], sess.epochs['maze2'][1]]
    )
# Constraining to epoch with times (start: 0.0, end: 1932.4200048116618)
# 20996/932598 total spikes spanning 40/64 units remain in subsession

active_subplots_shape = (1,1) # Single subplot
# active_subplots_shape = '1|2' # 1 subplot on left, two on right

## Get the config corresponding to this epoch/session settings:
# active_config = InteractivePlaceCellConfig(active_session_config=active_sess_config, active_epochs=active_epoch)
active_config = InteractivePlaceCellConfig(active_session_config=active_sess_config, active_epochs=active_epoch,
                        video_output_config=VideoOutputModeConfig(active_frame_range=np.arange(11070.0, 13970.0), video_output_parent_dir=Path('output', active_sess_config.session_name, active_epoch.name), active_is_video_output_mode=False),
                        plotting_config=PlottingConfig(output_subplots_shape=active_subplots_shape, output_parent_dir=Path('output', active_sess_config.session_name, active_epoch.name))) # '3|1    
## Looks like the only real outputs are: active_epoch_session_Neurons, active_epoch_pos
active_config.plotting_config.active_output_parent_dir.mkdir(parents=True, exist_ok=True) # makes the directory if it isn't already there

# Get the filtered and time-constrainted session and position data for this epoch:
active_epoch_session = sess.get_neuron_type('pyramidal').time_slice(active_epoch.t_start, active_epoch.t_stop)
print_subsession_neuron_differences(sess.neurons, active_epoch_session.neurons)
active_epoch_session_Neurons, active_epoch_pos, active_epoch_position_times = active_epoch_session.neurons, active_epoch_session.position, active_epoch_session.position.time

Constraining to units with type: pyramidal
Constraining to epoch with times (start: 0.0, end: 1932.4200048116618)
20996/932598 total spikes spanning 40/64 units remain in subsession


In [7]:
spikes_df = sess.spikes_df
# Get only the rows with a lap != -1:
# spikes_df = spikes_df[(spikes_df.lap != -1)] # 229887 rows × 13 columns
spikes_df # 932598 rows × 14 columns

,t,shank,cluster,aclu,qclu,cluinfo,x,y,speed,traj,lap,cell_type,t_seconds,flat_spike_idx
0,98,11,6,60,5,"(5, 5, 5, 5)",NaN,NaN,NaN,0,-1,NeuronType.CONTAMINATED,0.0098,0
1,161,8,13,53,4,"(0, 0, 4, 4)",NaN,NaN,NaN,0,-1,NeuronType.PYRAMIDAL,0.0161,1
2,285,11,6,60,5,"(5, 5, 5, 5)",NaN,NaN,NaN,0,-1,NeuronType.CONTAMINATED,0.0285,2
3,419,11,6,60,5,"(5, 5, 5, 5)",0.374502,0.338421,0.0,0,-1,NeuronType.CONTAMINATED,0.0419,3
4,422,7,5,41,5,"(5, 5, 5, 5)",0.374502,0.338421,0.0,0,-1,NeuronType.CONTAMINATED,0.0422,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
932593,64388837,9,3,56,5,"(5, 5, 5, 5)",NaN,NaN,NaN,0,-1,NeuronType.CONTAMINATED,6438.8837,932593
932594,64388904,1,9,6,5,"(5, 5, 5, 5)",NaN,NaN,NaN,0,-1,NeuronType.CONTAMINATED,6438.8904,932594
932595,64388972,3,8,30,5,"(4, 5, 5, 5)",NaN,NaN,NaN,0,-1,NeuronType.CONTAMINATED,6438.8972,932595
932596,64388972,7,5,41,5,"(5, 5, 5, 5)",NaN,NaN,NaN,0,-1,NeuronType.CONTAMINATED,6438.8972,932596


In [9]:
# Group by the lap column:
lap_grouped_spikes_df = spikes_df.groupby(['lap']) #  as_index=False keeps the original index
lap_spike_indicies = list()
for i in np.arange(sess.laps.n_laps):
    curr_lap_id = sess.laps.lap_id[i]
    #curr_flat_cell_indicies = (flat_spikes_out_dict['aclu'] == curr_cell_id) # the indicies where the cell_id matches the current one
    curr_lap_dataframe = lap_grouped_spikes_df.get_group(curr_lap_id)
    # curr_lap_spike_indicies = curr_lap_dataframe.flat_spike_idx.values
    lap_spike_indicies.append(curr_lap_dataframe.flat_spike_idx.values)
    # spiketrains.append(curr_cell_dataframe[time_variable_name].to_numpy())
    # shank_ids[i] = curr_cell_dataframe['shank'].to_numpy()[0] # get the first shank identifier, which should be the same for all of this curr_cell_id
    # cell_quality[i] = curr_cell_dataframe['qclu'].mean() # should be the same for all instances of curr_cell_id, but use mean just to make sure
    # cell_type.append(curr_cell_dataframe['cell_type'].to_numpy()[0])

lap_spike_indicies
# sess.laps.n_laps

[array([ 15841,  15842,  15843, ..., 900603, 900604, 900605], dtype=int64),
 array([ 31363,  31364,  31365, ..., 910916, 910917, 910918], dtype=int64),
 array([51216, 51217, 51218, ..., 68883, 68884, 68885], dtype=int64),
 array([68886, 68887, 68888, ..., 73854, 73855, 73856], dtype=int64),
 array([145897, 145898, 145899, ..., 154185, 154186, 154187], dtype=int64),
 array([162648, 162649, 162650, ..., 167232, 167233, 167234], dtype=int64),
 array([343909, 343910, 343911, ..., 359677, 359678, 359679], dtype=int64),
 array([369732, 369733, 369734, ..., 373584, 373585, 373586], dtype=int64),
 array([396149, 396150, 396151, ..., 404979, 404980, 404981], dtype=int64),
 array([412413, 412414, 412415, ..., 417038, 417039, 417040], dtype=int64),
 array([422331, 422332, 422333, ..., 430739, 430740, 430741], dtype=int64),
 array([442819, 442820, 442821, ..., 446546, 446547, 446548], dtype=int64),
 array([456992, 456993, 456994, ..., 461217, 461218, 461219], dtype=int64),
 array([466650, 466651, 

In [10]:
## Compute Placefields if needed:
from neuropy.analyses import Pf1D, Pf2D, perform_compute_placefields, plot_all_placefields
from neuropy.plotting.spikes import get_neuron_colors

should_force_recompute_placefields = True
should_display_2D_plots = False
# Setup the placefield computation config
active_config.computation_config = PlacefieldComputationParameters(speed_thresh=3, grid_bin=2, smooth=2)
# print_subsession_neuron_differences(sess.neurons, active_epoch_session_Neurons)

try: active_epoch_placefields1D
except NameError: active_epoch_placefields1D = None # Checks variable active_epoch_placefields's existance, and sets its value to None if it doesn't exist so it can be checked in the next step

try: active_epoch_placefields2D
except NameError: active_epoch_placefields2D = None # Checks variable active_epoch_placefields's existance, and sets its value to None if it doesn't exist so it can be checked in the next step

active_epoch_placefields1D, active_epoch_placefields2D = perform_compute_placefields(active_epoch_session_Neurons, active_epoch_pos, active_config.computation_config, active_epoch_placefields1D, active_epoch_placefields2D, should_force_recompute_placefields=True)
# Plot the placefields computed and save them out to files:
if should_display_2D_plots:
    ax_pf_1D, occupancy_fig, active_pf_2D_figures = plot_all_placefields(active_epoch_placefields1D, active_epoch_placefields2D, active_config)
else:
    print('skipping 2D placefield plots')
    
# Focus on the 2D placefields:
active_epoch_placefields = active_epoch_placefields2D
# Get the cell IDs that have a good place field mapping:
good_placefield_neuronIDs = np.array(active_epoch_placefields.ratemap.neuron_ids) # in order of ascending ID
print('good_placefield_neuronIDs: {}; ({} good)'.format(good_placefield_neuronIDs, len(good_placefield_neuronIDs)))

## Filter by neurons with good placefields only:
active_epoch_session = active_epoch_session.get_by_id(good_placefield_neuronIDs) # Filter by good placefields only, and this fetch also ensures they're returned in the order of sorted ascending index ([ 2  3  5  7  9 12 18 21 22 23 26 27 29 34 38 45 48 53 57])
active_epoch_session_Neurons = active_epoch_session.neurons
pf_sort_ind = np.array([int(i) for i in np.arange(len(active_epoch_session.neuron_ids))]) # convert to integer scalar array
pf_colors = get_neuron_colors(pf_sort_ind) # colors are by ascending index ID

active_config.plotting_config.pf_sort_ind = pf_sort_ind
active_config.plotting_config.pf_colors = pf_colors
active_config.plotting_config.active_cells_colormap = pf_colors.T # Make the colormap from the listed colors, used seemingly only by 'runAnalysis_PCAandICA(...)'
# self.active_config.plotting.active_cells_listed_colormap = ListedColormap(self.active_config.plotting.active_cells_colormap)


Recomputing active_epoch_placefields...
	 done.
Recomputing active_epoch_placefields2D...
	 done.
skipping 2D placefield plots
good_placefield_neuronIDs: [ 2  3  4  5  7  9 11 12 17 18 21 22 23 27 28 29 33 34 38 39 42 44 45 46
 47 48 53 55 57 58]; (30 good)


In [23]:
# Unpacking final values into separate variables:

def unpack_variables(active_session):
    # Spike variables: num_cells, spike_list, cell_ids, flattened_spikes
    num_cells = active_session.neurons.n_neurons
    spike_list = active_session.neurons.spiketrains
    cell_ids = active_session.neurons.neuron_ids
    # flattened_spikes = active_session.neurons.get_flattened_spikes() # get_flattened_spikes(..) returns a FlattenedSpiketrains object
    
    # reverse_cellID_idx_lookup_map = build_cellID_reverse_lookup_map(cell_ids)
    reverse_cellID_idx_lookup_map = active_session.neurons.reverse_cellID_index_map
    
    # Position variables: t, x, y
    t = active_session.position.time
    x = active_session.position.x
    y = active_session.position.y
    linear_pos = active_session.position.linear_pos
    speeds = active_session.position.speed 

    return num_cells, spike_list, cell_ids, reverse_cellID_idx_lookup_map, t, x, y, linear_pos, speeds
# # Spike variables:
# num_cells = active_epoch_session_Neurons.n_neurons
# spike_list = active_epoch_session_Neurons.spiketrains
# cell_ids = active_epoch_session_Neurons.neuron_ids
# flattened_spikes = active_epoch_session_Neurons.get_flattened_spikes() # get_flattened_spikes(..) returns a FlattenedSpiketrains object
# # Position variables: t, x, y
# t = active_epoch_pos.time
# x = active_epoch_pos.x
# y = active_epoch_pos.y
# linear_pos = active_epoch_pos.linear_pos
# speeds = active_epoch_pos.speed # note this has 1 less element than active_epoch_pos.x

num_cells, spike_list, cell_ids, reverse_cellID_idx_lookup_map, t, x, y, linear_pos, speeds = unpack_variables(active_epoch_session)
# Determine the x and y positions each spike occured for each cell
# spike_positions_list = build_spike_positions_list(spike_list, t, x, y)
# print('num_cells: {}'.format(num_cells))
# print('cell_ids: {}'.format(cell_ids)) # cell_ids is now a regular python list with 57 elements


# # Gets the flattened spikes, sorted in ascending timestamp for all cells.
# num_flattened_spikes = np.size(flattened_spikes.flattened_spike_times)
# print('num_flattened_spikes: {}'.format(num_flattened_spikes))
# # Build the Active UnitIDs
# flattened_spike_active_unitIdentities = np.array([int(reverse_cellID_idx_lookup_map[original_cellID]) for original_cellID in flattened_spikes.flattened_spike_identities]) # since flattened_spikes.flattened_spike_identities is already sorted, don't double sort
# ## Build the flattened spike positions list
# flattened_spike_positions_list = np.concatenate(tuple(spike_positions_list), axis=1) # needs tuple(...) to conver the list into a tuple, which is the format it expects
# flattened_spike_positions_list = flattened_spike_positions_list[:, flattened_spikes.flattened_sort_indicies] # ensure the positions are ordered the same as the other flattened items so they line up
# print('flattened_spike_positions_list: {}'.format(np.shape(flattened_spike_positions_list))) # (2, 19647)

# print(active_epoch_session.flattened_spiketrains.spikes_df)

# s_array = active_epoch_session.flattened_spiketrains.spikes_df[["x", "y"]].to_numpy()
# s_array
flattened_spike_positions_list = active_epoch_session.flattened_spiketrains.spikes_df[["x", "y"]].to_numpy().T
print('flattened_spike_positions_list: {}'.format(np.shape(flattened_spike_positions_list))) # (2, 19647)

flattened_spike_positions_list: (2, 19647)


In [24]:
# flattened_spikes.flattened_spike_identities # array([53, 45, 45, ..., 45, 22,  2])
# flattened_spikes.flattened_sort_indicies # array([16984, 13674, 13675, ..., 16068,  7575,   580], dtype=int64)
# flattened_spikes.flattened_spike_times # array([1.6100000e-02, 2.7230000e-01, 2.8300000e-01, ..., 1.9308899e+03, 1.9316915e+03, 1.9320706e+03])
# print(flattened_spike_positions_list) # [[103.5329515  100.94484811 100.86902587 ... 215.55358368 216.0612899 217.59276516], [100.64776532  98.26058916  98.16262888 ... 139.36778275 138.94465291 139.07649496]]
num_flattened_spikes # 19647

NameError: name 'num_flattened_spikes' is not defined

In [ ]:
active_spikes_df = active_epoch_session.neurons.spikes_df
# Get only the rows with a lap != -1:
active_spikes_df = active_spikes_df[(active_spikes_df.lap != -1)] # 229887 rows × 13 columns
active_spikes_df

In [25]:
## ICA and PCA Analysis
should_show_2D_ICA_plots = False
from PhoPositionalData.analysis.neuronal_dimensionality_reduction import runAnalysis_PCAandICA
active_session_ensembles, template, zsc_template, pca_data = runAnalysis_PCAandICA(active_epoch_session_Neurons, bin_size=0.250, frate_thresh=0.1, should_plot=should_show_2D_ICA_plots, active_cells_colormap=self.active_config.plotting_config.active_cells_colormap)

NameError: name 'self' is not defined

In [26]:
# have active_epoch_position_times: the actual times each position sample occured in seconds, active_epoch_relative_position_times: the same as active_epoch_position_times but starting at zero
# describe the movement

from PhoPositionalData.plotting.visualization_window import VisualizationWindow # Used to build "Windows" into the data points such as the window defining the fixed time period preceeding the current time where spikes had recently fired, etc.

# Split the position data into equal sized chunks to be displayed at a single time. These will look like portions of the trajectory and be used to animate. # Chunk the data to create the animation.
curr_plot_update_step = 1 # Update every frame
curr_plot_update_frequency = curr_plot_update_step * active_epoch_pos.sampling_rate # number of updates per second (Hz)
num_time_points = active_epoch_pos.n_frames / curr_plot_update_step
print('active_epoch_pos.sampling_rate (Hz): {}'.format(active_epoch_pos.sampling_rate))

# curr_window_duration = 2.5 # in seconds
# curr_view_window_length_samples = int(np.floor(curr_window_duration * active_epoch_pos.sampling_rate)) # number of samples the window should last
# recent_spikes_window = VisualizationWindow(duration_seconds=curr_window_duration, duration_num_frames=curr_view_window_length_samples)

# curr_recently_window_duration = 0.5 # in seconds
# curr_view_window_length_samples = int(np.floor(curr_window_duration * active_epoch_pos.sampling_rate)) # number of samples the window should last

## Simplified with just two windows:
longer_spikes_window = VisualizationWindow(duration_seconds=1024.0, sampling_rate=active_epoch_pos.sampling_rate) # have it start clearing spikes more than 30 seconds old
curr_view_window_length_samples = longer_spikes_window.duration_num_frames # number of samples the window should last
print('longer_spikes_window - curr_view_window_length_samples - {}'.format(curr_view_window_length_samples))

recent_spikes_window = VisualizationWindow(duration_seconds=1.0, sampling_rate=active_epoch_pos.sampling_rate)
curr_view_window_length_samples = recent_spikes_window.duration_num_frames # number of samples the window should last
print('recent_spikes_window - curr_view_window_length_samples - {}'.format(curr_view_window_length_samples))

## Build the sliding windows:
from numpy.lib.stride_tricks import sliding_window_view
# build a sliding window to be able to retreive the correct flattened indicies for any given timestep
active_epoch_position_linear_indicies = np.arange(np.size(active_epoch_position_times))
pre_computed_window_sample_indicies = recent_spikes_window.build_sliding_windows(active_epoch_position_linear_indicies)
# print('pre_computed_window_sample_indicies: {}\n shape: {}'.format(pre_computed_window_sample_indicies, np.shape(pre_computed_window_sample_indicies)))

## New Pre Computed Indicies Way:
z_fixed = np.full((recent_spikes_window.duration_num_frames,), 1.1)

active_epoch_pos.sampling_rate (Hz): 29.96976599553983
longer_spikes_window - curr_view_window_length_samples - 30689
recent_spikes_window - curr_view_window_length_samples - 29


## Main Spike/Placemap plotting:

In [27]:
from PhoPositionalData.plotting.gui import customize_default_pyvista_theme, print_controls_helper_text
customize_default_pyvista_theme() # Sets the default theme values to those specified in my imported file
# This defines the position of the vertical/horizontal splitting, in this case 40% of the vertical/horizontal dimension of the window
# pv.global_theme.multi_rendering_splitting_position = 0.40
pv.global_theme.multi_rendering_splitting_position = 0.80

from PhoPositionalData.plotting.camera_manipulation import * # apply_camera_view, apply_close_perspective_camera_view, apply_close_overhead_zoomed_camera_view
from PhoPositionalData.plotting.spikeAndPositions import build_active_spikes_plot_data, build_flat_map_plot_data, perform_plot_flat_arena, build_spike_spawn_effect_light_actor, spike_geom_circle, spike_geom_box, spike_geom_cone, animal_location_circle, animal_location_trail_circle
# from PhoPositionalData.plotting.spikeAndPositions import InteractiveSliderWrapper # for wrapping the slider

num_time_points = active_epoch_pos.n_frames / curr_plot_update_step
# print('num_time_points: {}\n'.format(num_time_points))

## Opacity Helpers:
last_only_opacity_values = np.zeros([curr_view_window_length_samples,])
last_only_opacity_values[-1] = 1.0
# gradually_fading_opacity_values = np.arange(curr_view_window_length_samples)
gradually_fading_opacity_values = np.linspace(0.0, 1.0, curr_view_window_length_samples)
long_gradually_fading_opacity_values = np.linspace(0.0, 1.0, longer_spikes_window.duration_num_frames)
sharply_fading_opacity_values = np.linspace(0.0, 0.6, curr_view_window_length_samples)
# sharply_fading_opacity_values[-1] = 0.1 # last element (corresponding to current position) is set to 1.0

# active_trail_opacity_values = last_only_opacity_values.copy()
# active_trail_opacity_values = gradually_fading_opacity_values.copy()
active_trail_opacity_values = sharply_fading_opacity_values.copy()
# print('active_trail_opacity_values: {}\n'.format(np.shape(active_trail_opacity_values)))
# active_trail_size_values = np.full([curr_view_window_length_samples,], 0.6) # all have a scale of 0.6
active_trail_size_values = np.linspace(0.2, 0.6, curr_view_window_length_samples) # fade from a scale of 0.2 to 0.6
# active_trail_size_values[-1] = 6.0 # except for the end (current) point, which has a scale of 1.0
# active_trail_size_values = sharply_fading_opacity_values.copy()

Applying custom Pyvista theme.
done.


# Plot Placemaps on 3D Plot

In [ ]:
from PhoPositionalData.plotting.spikeAndPositions import plot_placefields2D, update_plotVisiblePlacefields2D, build_custom_placefield_maps_lookup_table
from PhoPositionalData.plotting.gui import SetVisibilityCallback, MutuallyExclusiveRadioButtonGroup, add_placemap_toggle_checkboxes, add_placemap_toggle_mutually_exclusive_checkboxes

from PhoPositionalData.plotting.gui import CallbackSequence, SetUICheckboxValueCallback, OnUICheckboxChangedCallback
# # from PhoGui import vtk_ui
# # from PhoGui.vtk_ui.manager import get_manager, manager_exists, delete_manager
# from pyvista import _vtk
# # import vtk
# # noinspection PyUnresolvedReferences
# import vtkmodules.vtkInteractionStyle
# # noinspection PyUnresolvedReferences
# import vtkmodules.vtkRenderingOpenGL2
# from vtkmodules.vtkCommonColor import vtkNamedColors
# from vtkmodules.vtkCommonCore import vtkLookupTable
# from vtkmodules.vtkFiltersCore import vtkElevationFilter
# from vtkmodules.vtkFiltersModeling import vtkBandedPolyDataContourFilter
# from vtkmodules.vtkFiltersSources import vtkConeSource
# from vtkmodules.vtkRenderingCore import (
#     vtkActor,
#     vtkPolyDataMapper,
#     vtkRenderWindow,
#     vtkRenderWindowInteractor,
#     vtkRenderer
# )
# nc = vtkNamedColors()

# from pyvista.plotting.tools import parse_color, FONTS
# from PhoGui.PhoCustomVtkWidgets import PhoWidgetHelper
# from PhoGui.PhoCustomVtkWidgets import MultilineTextConsoleWidget

from PhoGui.InteractivePlotter.InteractivePlaceCellTuningCurvesDataExplorer import InteractivePlaceCellTuningCurvesDataExplorer

try: pActiveTuningCurvesPlotter
except NameError: pActiveTuningCurvesPlotter = None # Checks variable p's existance, and sets its value to None if it doesn't exist so it can be checked in the next step
ipcDataExplorer = InteractivePlaceCellTuningCurvesDataExplorer(active_config, x, y, active_epoch_placefields, pf_colors, extant_plotter=pActiveTuningCurvesPlotter)

In [29]:
pActiveTuningCurvesPlotter = ipcDataExplorer.plot(pActiveTuningCurvesPlotter)

No extant BackgroundPlotter
Creating a new BackgroundPlotter
done.


C:\Users\Pho\repos\NeuroPy\neuropy\utils\mathutil.py:23: RuntimeWarning: invalid value encountered in true_divide
  return (x - np.min(x, axis=axis, keepdims=True)) / np.ptp(


num_curr_tuning_curves: 30


In [33]:
## Slider with Callback Function Example:
import PhoGui
from PhoGui.InteractivePlotter.PhoInteractivePlotter import PhoInteractivePlotter
from PhoGui.InteractivePlotter.shared_helpers import InteractivePyvistaPlotterBuildIfNeededMixin
from PhoGui.InteractivePlotter.InteractivePlaceCellDataExplorer import InteractivePlaceCellDataExplorer

active_config.plotting_config.show_legend = True

try: pActiveInteractivePlaceSpikesPlotter
except NameError: pActiveInteractivePlaceSpikesPlotter = None # Checks variable p's existance, and sets its value to None if it doesn't exist so it can be checked in the next step
ipspikesDataExplorer = InteractivePlaceCellDataExplorer(active_config, active_epoch_session, t, x, y, num_time_points, extant_plotter=pActiveInteractivePlaceSpikesPlotter)
print('done.')

done.


[autoreload of neuropy.core.flattened_spiketrains failed: Traceback (most recent call last):
  File "C:\Users\Pho\anaconda3\envs\phoviz_test\lib\site-packages\IPython\extensions\autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "C:\Users\Pho\anaconda3\envs\phoviz_test\lib\site-packages\IPython\extensions\autoreload.py", line 394, in superreload
    module = reload(module)
  File "C:\Users\Pho\anaconda3\envs\phoviz_test\lib\imp.py", line 314, in reload
    return importlib.reload(module)
  File "C:\Users\Pho\anaconda3\envs\phoviz_test\lib\importlib\__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 613, in _exec
  File "<frozen importlib._bootstrap_external>", line 846, in exec_module
  File "<frozen importlib._bootstrap_external>", line 983, in get_code
  File "<frozen importlib._bootstrap_external>", line 913, in source_to_code
  File "<frozen importlib._bootstrap>", line 228, in _call_

In [31]:
pActiveInteractivePlaceSpikesPlotter = ipspikesDataExplorer.plot(pActivePlotter=pActiveInteractivePlaceSpikesPlotter)

NameError: name 'ipspikesDataExplorer' is not defined

In [ ]:

# ## Playback animation:
# # pl = PhoInteractivePlotter(pyvista_plotter=p)

# # ## Testing Updating the Timestamp Slider Programmatically:
# # interactive_timestamp_slider_wrapper.curr_index
# # interactive_timestamp_slider_wrapper.step_index(15)
        
# animate = InterfaceProperties(interactive_timestamp_slider_wrapper)
# # An unused constant-time callback that calls back every so often to perform updates
# p.add_callback(animate, interval=16)  # to be smooth on 60Hz

# # A checkbox that decides whether we're playing back at a constant rate or not.
# interactive_checkbox_actor = p.add_checkbox_button_widget(toggle_animation, value=False, color_on='green')

In [ ]:


centroid_point = np.asarray(centeroidnp(sess.position.traces.T))
print('centroid_point: {}'.format(centroid_point)) # (102.63442368751674, 102.63442368751674). (182.32292593117836, 139.47398492458092)
bounds = min_max_bounds(sess.position.traces.T)
print('bounds: {}'.format(bounds))
midpoint = np.asarray(bounds_midpoint(bounds))
print('midpoint: {}'.format(midpoint))

In [ ]:
# Plot the centroid point:
# p.add_points(centroid_point.T, color='blue')

# actor = p.add_points(np.append(centroid_point, [0.0]), render_points_as_spheres=True, point_size=10.0, color='cyan')
actor = p.add_points(np.append(midpoint, [0.0]), render_points_as_spheres=True, point_size=10.0, color='cyan')

In [ ]:
p.remove_actor(actor)

In [ ]:
# Camera:
def debug_print_camera(camera):
    print('camera: position: {}'.format(camera.position))
    
# camera = pv.Camera()
camera = p.camera
# print('camera: {}'.format(camera))
debug_print_camera(camera) # (419.82010625260347, 397.37481091874116, 278.0261261309909)
# top: (142.89398002624512, 120.44868469238281, 480.75012047191734)
# pl.camera.position

In [ ]:
camera.position = np.append(midpoint, [0.0])